In [1]:
!pip install torchsummary
!pip install av
!pip install pytorchvideo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 858.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188714 sha256=a3e7d8beba43e4dddc69e9325ad3702e8a3042db6c4c10f80ba38cdeae204b78
  Stored in directory: /root/.cache/pip/wheels/e8/51/05/053b29bac2400cbbae2fb7cfc41afd280d627bca7c9363ca80
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=c1e68e60cd2e7ca207a97a927eea0fe2ca8125c0eb3a5f7102daba4e4df7d3c8
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
Successfully built pytorchvideo fvcore


In [2]:
start= 0
end = 3

In [3]:
import torch
import numpy as np
from torch.utils.data import (
    Dataset,
    DataLoader,
) 
import pickle


import os

import math
import torch.nn as nn
import torch.nn.functional as F

from torchvision.transforms import Compose, Lambda, Grayscale,Normalize, CenterCrop,Resize,ConvertImageDtype,ToTensor
from torchvision.io import read_video
from tqdm import tqdm

from collections import OrderedDict

import torch.optim as optim

from torch.autograd import Variable


import torchvision.models as models
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
)

import sys
from torchsummary import summary






class SignDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y


    def __len__(self):

        return len(self.y)

        # length = 0
        # with open(self.file, 'rb') as f:
        #     data = pickle.load(f)

        # for x in data:
        #     length += len(data[x])
        # return length


    def __getitem__(self, index):
        return self.x[index], self.y[index]
        

In [4]:
IMAGE_HEIGHT = 720
IMAGE_WIDTH = 800
IMAGE_CHANNEL = 1
NUM_FRAMES = 25
NUM_CLASSES = 60






def transform_data(x, mean, std):
    
    transform =  ApplyTransformToKey(
        key="video",
        transform=Compose(
            [
                Lambda(lambda x: x/255.0),
                
                Normalize((mean), (std)),

                CenterCrop([720,900]),
                Resize([512,512]),
                Lambda(lambda x: x.permute(1,2,3,0)),#(channel, frames(depth), height, width)

            ]

        ),
    )
    
    return transform(x)

def lp_video(video,start_time, end_time):
    video_data = video.get_clip(start_sec=float(start_time)/1000.0, end_sec=float(end_time)/1000.0)
            #print(video_data["video"].shape)
 
    if video_data["video"] is None:
        return None        # or pass
    else:
    

    #             video_data["video"] = video_data["video"]/255
                #print(video_data["video"].shape)
        video_data["video"] = video_data["video"].permute(1,0,2,3)
        std, mean = torch.std_mean(video_data["video"], dim = [0,2,3])
        std = std/255.0
        mean = mean/255.0
        video_data = transform_data( video_data, mean, std)

        return video_data["video"]

def load_dataloaders(batch_size,start,end):

    time_start = []
    time_end = []
    train_inputs =[] #x
    train_classes = [] #y

    with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
        data = pickle.load(f)


# keys are files so iterate only limited files due to memory limitations.
    for key in list(data.keys())[start:end]:
        filename = key
        print("file",filename)
        video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4")
    # file functions

        for x in data[key]:
            img_cls = x[0]
            time_start.append(x[1])
            time_end.append(x[2])
            
            
            # start_time = x[1]
            # end_time = x[2]
            vid = lp_video(video,x[1], x[2])

            for m in torch.unbind(vid, dim=3):
                train_classes.append(img_cls)
                train_inputs.append(m)


            #some negative classes too
        for i in range(len(time_start)-1):
            if (time_start[i+1]- time_end[i])>1000.0:
                start_t = time_end[i]+700.0
                end_t = time_end[i]+750.0
                vid = lp_video(video,start_t, end_t)
                if vid is None:
                    break
                else:
                    for m in torch.unbind(vid, dim=3):
                        train_classes.append(60)
                        train_inputs.append(m)

            
        
            

    signds = SignDataset(train_inputs, train_classes)
    trainlen = int(len(signds)*0.8)
    torch.manual_seed(0)
    
    train_set, val_test_set = torch.utils.data.random_split(signds, [trainlen, len(signds)-trainlen])
    trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

    valloader = DataLoader(val_test_set, batch_size=batch_size, shuffle=True, num_workers=2)
  
    return trainloader, valloader
        
    
   
    

In [5]:
# IMAGE_HEIGHT = 512
# IMAGE_WIDTH = 512
# IMAGE_CHANNEL = 1
# NUM_FRAMES = 10
# NUM_CLASSES = 61



# train_inputs =[] #x
# train_classes = [] #y

# def preprocess(batch,mean,std):
#     transforms = Compose(
#         [
            
#             Lambda(lambda x: x/255.0),
#             CenterCrop(size=(720, 900)),
            
#             Normalize(mean=mean, std=std),  # map [0, 1] into [-1, 1]
#             Grayscale(num_output_channels=1),
#             Resize(size=(512, 512)),
#         ]
#     )
#     batch = transforms(batch)
#     return batch



# def lp_video(filename,start_time, end_time):
#     frames, _, _ = read_video("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4",start_pts=start_time/1000.0,end_pts= end_time/1000.0,pts_unit='sec')
    
#     frames = frames.permute(0, 3, 1, 2)  # (N, H, W, C) -> (N, C, H, W)
#     frames = ConvertImageDtype(torch.float32)(frames)
    

    
  
            
#     std, mean = torch.std_mean(frames)
#     std = std/255.0
#     mean = mean/255.0

#     # tList = [m for m in torch.unbind(video_data, dim=4) ]
#     # y = torch.stack(tList, dim=0)
        
            
            
#     img1_batch = preprocess( frames, mean, std)
    
#     return img1_batch

# def load_dataloaders(batch_size,start,end):

#     time_start = []
#     time_end = []

#     with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
#         data = pickle.load(f)


# # keys are files so iterate only limited files due to memory limitations.
#     for key in list(data.keys())[start:end]:
        
#         print(" \n file:",key)
    
#         for x in data[key]:
#             img_cls = x[0]
#             time_start.append(x[1])
#             time_end.append(x[2])
            
            
#             # start_time = x[1]
#             # end_time = x[2]
            
#             vid = lp_video(key,x[1], x[2])

#             for m in vid:
#                 train_classes.append(img_cls)
#                 train_inputs.append(m)


#             #some negative classes too
#         for i in range(0,5):
#             if  time_start[i+1]-time_end[i]>800:
#                 start_t = time_end[i]+500
#                 end_t = time_end[i]+550
#                 vid = lp_video(key,start, end)
#                 for m in vid:
#                     print("here")
#                     train_classes.append(60)
#                     train_inputs.append(m)

            
        
            

#     signds = SignDataset(train_inputs, train_classes)
#     trainlen = int(len(signds)*0.8)
#     torch.manual_seed(0)
    
#     train_set, val_test_set = torch.utils.data.random_split(signds, [trainlen, len(signds)-trainlen])
#     trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

#     valloader = DataLoader(val_test_set, batch_size=batch_size, shuffle=True, num_workers=2)
  
#     return trainloader, valloader
        
    

In [6]:
"""
Creates a EfficientNetV2 Model as defined in:
Mingxing Tan, Quoc V. Le. (2021). 
EfficientNetV2: Smaller Models and Faster Training
arXiv preprint arXiv:2104.00298.
import from https://github.com/d-li14/mobilenetv2.pytorch
"""



__all__ = ['effnetv2_s', 'effnetv2_m', 'effnetv2_l', 'effnetv2_xl']


def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


# SiLU (Swish) activation function
if hasattr(nn, 'SiLU'):
    SiLU = nn.SiLU
else:
    # For compatibility with old PyTorch versions
    class SiLU(nn.Module):
        def forward(self, x):
            return x * torch.sigmoid(x)

 
class SELayer(nn.Module):
    def __init__(self, inp, oup, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
                nn.Linear(oup, _make_divisible(inp // reduction, 8)),
                SiLU(),
                nn.Linear(_make_divisible(inp // reduction, 8), oup),
                nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y


def conv_3x3_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


class MBConv(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, use_se):
        super(MBConv, self).__init__()
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.identity = stride == 1 and inp == oup
        if use_se:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                SELayer(inp, hidden_dim),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # fused
                nn.Conv2d(inp, hidden_dim, 3, stride, 1, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )


    def forward(self, x):
        if self.identity:
            return x + self.conv(x)
        else:
            return self.conv(x)


class EffNetV2(nn.Module):
    def __init__(self, cfgs, num_classes=61, width_mult=1.):
        super(EffNetV2, self).__init__()
        self.cfgs = cfgs

        # building first layer
        input_channel = _make_divisible(24 * width_mult, 8)
        layers = [conv_3x3_bn(3, input_channel, 2)]
        # building inverted residual blocks
        block = MBConv
        for t, c, n, s, use_se in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 8)
            for i in range(n):
                layers.append(block(input_channel, output_channel, s if i == 0 else 1, t, use_se))
                input_channel = output_channel
        self.features = nn.Sequential(*layers)
        # building last several layers
        output_channel = _make_divisible(1792 * width_mult, 8) if width_mult > 1.0 else 1792
        self.conv = conv_1x1_bn(input_channel, output_channel)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(output_channel, num_classes)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.001)
                m.bias.data.zero_()


def effnetv2_s(**kwargs):
    """
    Constructs a EfficientNetV2-S model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  2, 1, 0],
        [4,  48,  4, 2, 0],
        [4,  64,  4, 2, 0],
        [4, 128,  6, 2, 1],
        [6, 160,  9, 1, 1],
        [6, 256, 15, 2, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_m(**kwargs):
    """
    Constructs a EfficientNetV2-M model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  3, 1, 0],
        [4,  48,  5, 2, 0],
        [4,  80,  5, 2, 0],
        [4, 160,  7, 2, 1],
        [6, 176, 14, 1, 1],
        [6, 304, 18, 2, 1],
        [6, 512,  5, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_l(**kwargs):
    """
    Constructs a EfficientNetV2-L model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  7, 2, 0],
        [4,  96,  7, 2, 0],
        [4, 192, 10, 2, 1],
        [6, 224, 19, 1, 1],
        [6, 384, 25, 2, 1],
        [6, 640,  7, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_xl(**kwargs):
    """
    Constructs a EfficientNetV2-XL model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  8, 2, 0],
        [4,  96,  8, 2, 0],
        [4, 192, 16, 2, 1],
        [6, 256, 24, 1, 1],
        [6, 512, 32, 2, 1],
        [6, 640,  8, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)

In [7]:
def evaluate(outputs: Variable, labels: Variable) -> float:
    """Evaluate neural network outputs against non-one-hotted labels."""
    Y = labels.numpy()
    Yhat = np.argmax(outputs, axis=1)
    return float(np.sum(Yhat == Y))

In [8]:

def test(model, device, loader, mode="Validate"):
    
    
    model.eval()

    correct = 0
    num_samples = 0
    
        
    for batch_idx, (data, target) in enumerate(loader):
        with torch.no_grad():    
            data = data.to(device)
            target = (target).to(device)

            output = model(data.half())  
            
            _,pred = output.max(1)# get the index of the max log-probability
            num_samples += target.size(0)
            correct += (pred==target).sum().item()
            #print("correct", correct)

    acc = 100.0 * correct / num_samples
    
    print('{} Accuracy: {}/{} ({:.0f}%)'.format(
                mode,correct, num_samples,
                100. * correct / num_samples, acc))

In [9]:
def trainonval(model, device,validloader, optimizer, criterion, epochs):

    
    model.train()

    
    
    print("Training on Validation Set starts with lr",optimizer.param_groups[0]['lr'])
    for epoch in range(epochs):
        correct = 0
        num_samples = 0
        train_loss = 0
        
        for batch_idx, (data, target) in enumerate(validloader):
            
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(data.half())
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            
            train_loss += loss.item()
            
            _,pred = output.max(1)# get the index of the max log-probability
            num_samples += target.size(0)
            correct += (pred==target).sum().item()
            
            
            
        
        #print(scheduler.get_last_lr())
        train_loss /= num_samples
        
        if (100 * correct / num_samples) >= 90:
            
            print('Epoch: {} , Training Accuracy on Val set: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,100. * correct / num_samples, train_loss))
            test(model, device, testloader, mode = "Test after Training on validation set")
                
            return model, optimizer
            
    
        print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,
                100. * correct / num_samples, train_loss))
        
    return model, optimizer

In [10]:

    


def train(model, device, train_loader,validloader,scheduler, optimizer,criterion, epochs):
    print("Train start")
    breakout = False
#     model.half()
#     model.cuda()

    
  

    model.train()

    
    
    for epoch in range(epochs):
        correct = 0
        num_samples = 0
        train_loss = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(data.half())
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            
            train_loss += loss.item()
            
            
            _,pred = output.max(1)# get the index of the max log-probability
            num_samples += target.size(0)
            correct += (pred==target).sum().item()
            
        #if optimizer.param_groups[0]['lr'] == 1e-2:    
        scheduler.step()
        #print("lr:",optimizer.param_groups[0]['lr'])
        
        
        train_loss /= num_samples
        
        if (100 * correct / num_samples) >= 95:
            
            print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,100. * correct / num_samples, train_loss))
            
            test(model,device,validloader, mode = "Validating")
#             test(model, device, testloader, mode = "Test before Training on validation set")
           # model, optimizer = trainonval(model, device, validloader, optimizer, criterion, epochs)
#             test(model, device, testloader, mode = "Test after training on validation set")
            
            
            
            torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "./model_optimizer1.pt")
            
            breakout = True
            print("breakout")
            return
            
            
            
       
        

        print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,
                100. * correct / num_samples, train_loss))
        if (((epoch+1)%5) == 0) :
            test(model,device,validloader)
            model.train()
            
    if breakout:
            print("breakout")
            return
    
    
    
    #model, optimizer = trainonval(model, device, validloader, optimizer, criterion, epochs)
    test(model, device, validloader, mode= "test set ")
    
    
        
    torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "./model_optimizer1.pt")
    
    
        


In [11]:
n_classes = 61


model = nn.Sequential(OrderedDict([
    ('frontend',effnetv2_s()),
]))





# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer and learning rate
optimizer = optim.SGD(
    [
        
        {"params": model.frontend.parameters(), "lr": 1e-1},
  ],
  momentum = 0.9
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
state = torch.load("../input/sign-classification/model_optimizer1.pt")
model.load_state_dict(state['model_state_dict'])
model.half()
model.cuda()
optimizer.load_state_dict(state['optimizer_state_dict'])

In [12]:

trainloader, valloader = load_dataloaders(batch_size=32,start=12, end=18)#73 not included 


file p04_n089
file p01_n044
file p05_n037
file p04_n038
file p06_n055
file p01_n099


In [13]:
#print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))

In [14]:
#summary(model, (3, 512, 512), device='cpu')

In [15]:
train(model,device,trainloader,valloader,scheduler,optimizer,criterion,50)

Train start
Epoch: 1 , Training Accuracy: 500/1388 (36%) Training Loss: 0.132620
Epoch: 2 , Training Accuracy: 849/1388 (61%) Training Loss: 0.053985
Epoch: 3 , Training Accuracy: 1014/1388 (73%) Training Loss: 0.036316
Epoch: 4 , Training Accuracy: 1131/1388 (81%) Training Loss: 0.025177
Epoch: 5 , Training Accuracy: 1184/1388 (85%) Training Loss: 0.019105
Validate Accuracy: 292/348 (84%)
Epoch: 6 , Training Accuracy: 1242/1388 (89%) Training Loss: 0.014561
Epoch: 7 , Training Accuracy: 1279/1388 (92%) Training Loss: 0.010795
Epoch: 8 , Training Accuracy: 1333/1388 (96%) Training Loss: 0.007712
Validating Accuracy: 313/348 (90%)
breakout


In [16]:
#from 21 15 epochs